## 使用KNN实现分类

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

iris = load_iris()
X = iris.data[:,:2]
y = iris.target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
# n_neighbors：邻居的数量。
# weights：权重计算方式。可选值为uniform与distance。
# uniform：所有样本统一权重。
# distance：样本权重与距离成反比。
knn = KNeighborsClassifier(n_neighbors=3,weights="uniform")
knn.fit(X_train,y_train)
y_hat = knn.predict(X_test)
print(classification_report(y_test,y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.78      0.44      0.56        16
           2       0.44      0.78      0.56         9

    accuracy                           0.71        38
   macro avg       0.74      0.74      0.71        38
weighted avg       0.77      0.71      0.71        38



## 超参数调整
网格交叉验证法

In [13]:
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
# 定义需要尝试的超参数组合
grid = {"n_neighbors":range(1,11,1),"weights":['uniform','distance']}
# estimator：评估器，即对哪个模型调整超参数。
# param_grid：需要检验的超参数组合。从这些组合中，寻找效果最好的超参数组合。
# scoring：模型评估标准。
# n_jobs：并发数量。
# cv：交叉验证折数。
# verbose：输出冗余信息，值越大，输出的信息越多。
gd = GridSearchCV(estimator=knn,param_grid=grid,scoring="accuracy",n_jobs=-1,cv=4,verbose=10)
gd.fit(X_train,y_train)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1839s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  58 out of  80 | elapsed:    3.6s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  67 out of  80 | elapsed:    3.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  76 out of  80 | elapsed:    3.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    3.6s finished


GridSearchCV(cv=4, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(1, 11),
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=10)

In [14]:
# 最好的分值
print(gd.best_score_)
# 最好的超参数组合
print(gd.best_params_)
# 使用最好的超参数训练模型
print(gd.best_estimator_)

0.7946428571428571
{'n_neighbors': 9, 'weights': 'uniform'}
KNeighborsClassifier(n_neighbors=9)


In [15]:
estimator = gd.best_estimator_
y_hat = estimator.predict(X_test)
print(classification_report(y_test,y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.75      0.38      0.50        16
           2       0.41      0.78      0.54         9

    accuracy                           0.68        38
   macro avg       0.72      0.72      0.68        38
weighted avg       0.76      0.68      0.68        38



## 使用KNN回归预测

In [16]:
from sklearn.datasets import load_boston
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.linear_model import LinearRegression

X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
knn = KNeighborsRegressor(n_neighbors=3, weights="uniform") 
knn.fit(X_train, y_train)
print("KNN算法R^2值：", knn.score(X_test, y_test))
lr = LinearRegression() 
lr.fit(X_train, y_train)
print("线性回归算法R^2值：", lr.score(X_test, y_test))


KNN算法R^2值： 0.509785478689029
线性回归算法R^2值： 0.6354638433202122


## 数据标准化
我们发现，使用KNN算法进行回归，效果比线性回归要差很多，但这并不能证明KNN算法是不如线性回 归的。原因在于，线性回归在训练参数时，不是基于距离进行计算的，因此，即使线性回归各个特征的 量纲（数量级）不同，也不影响最终的拟合效果（权重会不同）。不过，KNN是基于距离计算的，如果 特征之间的量纲不同，在计算时，量纲较大的特征就会占据主导地位，从而算法会忽略量纲较小的特征，这将会对模型性能造成较大的影响。

实际上，不只是KNN算法，很多算法对特征的数量级都是敏感的，因此，在使用算法之前，我们最好将 数据集中的特征转换成相同的量纲，从而消除不同量纲对算法造成的负面影响，我们将这个过程称为数 据标准化。实际上，即使特征量纲相同，标准化也不会产生负面影响。

常用的标准化方式为均值标准差标准化（StandardScaler）与最小最大值标准化
（MinMaxScaler）。


In [19]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = [StandardScaler(),MinMaxScaler()]
desc = ["均值标准差标准化","最小最大值标准化"]
for s,d in zip(scaler,desc):
    X_train_scale = s.fit_transform(X_train)
    X_test_scale = s.transform(X_test)
    knn = KNeighborsRegressor(n_neighbors=3,weights="uniform")
    knn.fit(X_train_scale,y_train)
    print(d,knn.score(X_test_scale,y_test))
 

均值标准差标准化 0.6248800677762865
最小最大值标准化 0.6177749492293981


## 流水线
在上例中，我们使用标准化对训练数据进行了转换，然后使用KNN模型对象在转换后的数据上进行拟合。可以说，这是两个步骤。我们虽然可以分别去执行这两个步骤，然而，当数据预处理的工作较多时，可能会涉及更多的步骤（例如多项式扩展，One-Hot编码，特征选择等操作），此时分别执行每个步骤会显得过于繁琐。

流水线（Pipeline类）可以将每个评估器视为一个步骤，然后将多个步骤作为一个整体而依次执行，这 样，我们就无需分别执行每个步骤。例如，在上例中，我们就可以将数据标准化与训练模型两个步骤视 为一个整体，一并执行。

流水线具有最后一个评估器的所有方法。当通过流水线对象调用方法   时，会执行这样的过程（假设流水线具有n个评估器）：
1、如果是fit方法，则会首先对前 n - 1个评估器依次调用fit_transform方法，然后在最后一个评估器上调用 （fit）方法。
2、如果是其他方法，则会首先对前 n - 1个评估器依次调用transform方法，然后在最后一个评估器上调用 f方法。

例如，当在流水线上调用fit方法时，将会依次在每个评估器上调用fit_transform方法，前一个评估器将 转换之后的结果传递给下一个评估器，直到最后一个评估器调用fit方法为止（最后一个评估器不会调用transform方法）。而当在流水线上调用predict方法时，则会依次在每个评估器上调用transform方
法，在最后一个评估器上调用predict方法。


In [23]:
from sklearn.pipeline import Pipeline

X,y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
#   定义流水线的步骤。类型为一个列表，列表中的每个元素是元组类型，
# 格式为：[(步骤名1，评估器1), (步骤名2， 评估器2)，……， (步骤名n, 评估器n) 
steps = [("scaler", StandardScaler()), ("knn", KNeighborsRegressor())] 
p = Pipeline(steps)
#   设置流水线的参数。所有可用的参数，可以通过get_params获取。
p.set_params(knn__n_neighbors=3, knn__weights="uniform") 
p.fit(X_train,y_train)
print(p.score(X_test,y_test))

0.6248800677762865
